## Load in the dataset CSV

## Make one column with classification

This is the subspecies (which may is a mix of two for some specimens, the hybrids); this is for coloring the distribution plot by hybrid status and showing all together.

## Get distribution of images by subspecies (colored by hybrid status)

## Start here

In [ ]:
import os
print(os.getcwd())

/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook')
print(os.getcwd())

/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook


In [ ]:
import os
print(os.listdir('../DINO_train'))
print(os.getcwd())

['model_utils.py', 'dataset.py', 'classifier.py', 'evaluation.py', '__pycache__', 'training.py', 'data_utils.py', 'fine_tune.py', '.ipynb_checkpoints']
/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook


## Import training requirements

In [ ]:
import torch
import sys
import os
import csv
from pathlib import Path
# Add your path of library
sys.path.append('../DINO_train')
import training

In [ ]:
import importlib
import training  # 替換為你的模組名稱

importlib.reload(training)

<module 'training' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/training.py'>

In [ ]:
import data_utils  # 替換為你的模組名稱

importlib.reload(data_utils)

<module 'data_utils' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/data_utils.py'>

## Model setup


In [ ]:
training.DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
model = training.get_dino_model(dino_name='facebook/dinov2-base').to(training.DEVICE)
training.BATCH_SIZE = 4

# Location to save checkpoints and results
training.CLF_SAVE_DIR = Path('./trained_clfs')
os.makedirs(training.CLF_SAVE_DIR, exist_ok=True)
print(training.DEVICE)
# print(model)
print(type(model))


cuda
<class 'transformers.models.dinov2.modeling_dinov2.Dinov2Model'>


## Start training

In [ ]:
img_path = "/content/drive/MyDrive/CWL_butterfly/Hackathon-Butterfly-Hybrid-Detection-main/Hackathon-Butterfly-Hybrid-Detection-main/input_data/New_both2"
data_path = "/content/drive/MyDrive/CWL_butterfly/Hackathon-Butterfly-Hybrid-Detection-main/Hackathon-Butterfly-Hybrid-Detection-main/input_data/0130_final.csv"

In [ ]:
train_data, test_data = training.load_data(data_path, img_path, test_size=0.2)

Filtering bad urls: 1824it [00:07, 251.89it/s]

Bad rows: 0


In [ ]:
import importlib
import training  # 替換為你的模組名稱

importlib.reload(training)

<module 'training' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/training.py'>

In [ ]:
import classifier
importlib.reload(classifier)

<module 'classifier' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/classifier.py'>

In [ ]:
import data_utils
importlib.reload(data_utils)

<module 'data_utils' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/data_utils.py'>

In [ ]:
import dataset
importlib.reload(dataset)

<module 'dataset' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/dataset.py'>

In [ ]:
# Create dataloader
tr_sig_dloader, test_dl = training.prepare_data_loaders(train_data, test_data, img_path)
print(training.data_transforms())

Created base dataset with 1459 samples
Created base dataset with 365 samples
Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomApply(
    p=0.5
    ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3), saturation=(0.7, 1.3), hue=(-0.1, 0.1))
)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    RandomHorizontalFlip(p=0.5)
    Lambda()
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Extract visual features from model
try:
    tr_features, tr_labels, test_features, test_labels = training.extract_features(tr_sig_dloader, test_dl, model)
    print(tr_features)
except Exception as e:
    print("Error in extracting features")
    print(e)

Extracting features: 100%|██████████| 92/92 [02:07<00:00,  1.38s/it]

[[-3.590167    1.0381408  -2.4874203  ... -0.73921764 -0.61626506
   0.01347281]
 [-1.8695996   0.48228502 -3.6843662  ... -1.0912731  -0.8381891
  -0.56001115]
 [-3.9477394   1.032649   -2.8307102  ... -0.5798567  -0.5941294
  -0.58015746]
 ...
 [-2.3476636   0.32665512 -2.1043124  ... -1.0751843  -0.65282476
  -0.44465563]
 [-3.9014523  -0.6913454  -1.9329932  ... -1.0465872  -1.1669434
  -0.05950663]
 [-1.4707919   1.2509     -2.3430572  ... -0.83313364 -0.38270462
  -0.43783084]]


In [ ]:
print(tr_features)
print(tr_labels)
print(test_features)
print(test_labels)

[[-3.590167    1.0381408  -2.4874203  ... -0.73921764 -0.61626506
   0.01347281]
 [-1.8695996   0.48228502 -3.6843662  ... -1.0912731  -0.8381891
  -0.56001115]
 [-3.9477394   1.032649   -2.8307102  ... -0.5798567  -0.5941294
  -0.58015746]
 ...
 [-2.3476636   0.32665512 -2.1043124  ... -1.0751843  -0.65282476
  -0.44465563]
 [-3.9014523  -0.6913454  -1.9329932  ... -1.0465872  -1.1669434
  -0.05950663]
 [-1.4707919   1.2509     -2.3430572  ... -0.83313364 -0.38270462
  -0.43783084]]
[0 0 0 ... 0 0 0]
[[-4.385366    0.8409648  -1.8811343  ... -1.0504885  -0.8694573
  -0.40624627]
 [ 0.6799864  -0.06571534 -0.6935987  ... -1.3331792  -0.3479464
  -0.18167572]
 [-3.1644666  -0.18131393 -2.4950194  ... -0.7554871  -0.57420623
  -0.67116976]
 ...
 [-3.0317197   0.9362211  -3.3507428  ... -0.9498002  -0.41591516
  -1.0083861 ]
 [-2.961629    0.82279193 -3.3791797  ... -1.4129974  -0.88604593
  -0.50875   ]
 [-3.2273757   0.5677382  -1.5304294  ... -1.0315692  -1.0201578
  -0.47204036]]
[0 0

In [ ]:
import importlib
import classifier
importlib.reload(classifier)
importlib.reload(training)

<module 'training' from '/content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/../DINO_train/training.py'>

In [ ]:
csv_output, score_output, scores_dict = training.train_and_evaluate(tr_features, tr_labels, test_features, test_labels)


Training and evaluating svm...
Saved svm classifier to /content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/clffff/trained_svm_classifier.pkl
svm: Acc - 0.9692, Hacc - 0.6800, NHacc - 0.9963

          Hybrid-Recall: 0.8333333333333334
          Hybrid-Precision: 0.46875
          f1-Score: 0.6
          ROC AUC: 0.8921709894332374
          Accuracy: 0.9452054794520548
          
Training and evaluating sgd...
Saved sgd classifier to /content/drive/MyDrive/fine_tune/CWL/Hackathon-Butterfly-Hybrid-Detection-Main-20250130T134851Z-001/Hackathon-Butterfly-Hybrid-Detection-Main/DINO_notebook/clffff/trained_sgd_classifier.pkl
sgd: Acc - 0.9589, Hacc - 0.5200, NHacc - 1.0000

          Hybrid-Recall: 1.0
          Hybrid-Precision: 0.5142857142857142
          f1-Score: 0.6792452830188679
          ROC AUC: 0.9755043227665706
          Accuracy: 0.9534246575342465
          
Training and ev

In [ ]:
# Save evaluation results
csv_filename = training.CLF_SAVE_DIR / "classifier_evaluation_results.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Configuration", "AUC", "Precision", "Recall", "F1-score"])
    writer.writerows(csv_output)

# Save individual scores
scores_filename = training.CLF_SAVE_DIR / "classifier_scores.csv"
with open(scores_filename, mode='w', newline='') as score_file:
    score_writer = csv.writer(score_file)
    score_writer.writerow(["Index", "Score", "True Label"])
    score_writer.writerows(score_output)